In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

In [ ]:
def dfc_to_graph(dfc_matrix, label):
    """
    Convert a dFC matrix into a fully connected weighted graph.
    """
    num_nodes = dfc_matrix.shape[0]

    # Node features: all nodes get identical 1D feature (e.g., [[1.0], [1.0], ..., [1.0]])
    x = torch.ones((num_nodes, 1), dtype=torch.float)

    # Create edge_index for all pairs (excluding self-loops if desired)
    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t()

    # Get corresponding edge weights from the dFC matrix
    edge_weights = dfc_matrix[edge_index[0], edge_index[1]]

    return Data(x=x, edge_index=edge_index, edge_attr=edge_weights, y=torch.tensor([label], dtype=torch.float))
